# MLflow x SQLite (Local Database Edition)

### Set up environment - connect to mlflow

In [1]:
%env MLFLOW_TRACKING_URI=sqlite:///mlruns.db

env: MLFLOW_TRACKING_URI=sqlite:///mlruns.db


**Alternatively, you can also perform it in your CLI as such**
`export MLFLOW_TRACKING URI=sqlite:///mlruns.db`

**If you are using window**


`set MLFLOW_TRACKING_URI=sqlite:///mlrun.db` 

*Noted that the 'set' function may not function sometimes, a better alternative may be to use*

`$env:MLFLOW_TRACKING_URI="sqlite:///mlrun.db`

### Set up example experiments

In [ ]:
# First example experiment

import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.sklearn.autolog()

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
rf.fit(X_train, y_train)

# Use the model to make predictions on the test dataset.
predictions = rf.predict(X_test)

In [16]:
# Second example experiment
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

mlflow.set_experiment("iris classification")

import os
# Start an MLflow run
with mlflow.start_run(run_name='abcde'):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
        
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path='iris_model',
        signature=signature,
        input_example=X_train,
        registered_model_name="iris_model",
    )

2024/01/08 10:06:39 INFO mlflow.tracking.fluent: Experiment with name 'iris classification' does not exist. Creating a new experiment.
C:\Users\carnot\anaconda3\envs\mlflow_testing\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'iris_model'.
2024/01/08 10:06:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model, version 1
Created version '1' of model 'iris_model'.


### Access to mlflow ui

`mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db`

## Brief look at the structure inside the database

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('mlruns.db')

In [5]:
c = conn.cursor()

### Display all tables in the database

In [19]:
query = """
SELECT name
FROM sqlite_master WHERE type='table';
"""
result = c.execute(query)
result.fetchall()

[('experiments',),
 ('alembic_version',),
 ('experiment_tags',),
 ('tags',),
 ('registered_models',),
 ('runs',),
 ('registered_model_tags',),
 ('model_version_tags',),
 ('model_versions',),
 ('latest_metrics',),
 ('metrics',),
 ('params',),
 ('registered_model_aliases',),
 ('datasets',),
 ('inputs',),
 ('input_tags',)]

### Brief touch into what each table contains

#### experiments

In [26]:
# table structure
query = """
PRAGMA table_info('experiments')
"""
result = c.execute(query)
result.fetchall()

[(0, 'experiment_id', 'INTEGER', 1, None, 1),
 (1, 'name', 'VARCHAR(256)', 1, None, 0),
 (2, 'artifact_location', 'VARCHAR(256)', 0, None, 0),
 (3, 'lifecycle_stage', 'VARCHAR(32)', 0, None, 0),
 (4, 'creation_time', 'BIGINT', 0, None, 0),
 (5, 'last_update_time', 'BIGINT', 0, None, 0)]

In [20]:
# content from the two example experiments
query = """
SELECT * 
FROM experiments
"""
result = c.execute(query)
result.fetchall()

[(0,
  'Default',
  'file:///C:/Users/carnot/Desktop/mlflow/mlruns/0',
  'active',
  1704441364229,
  1704441364229),
 (1,
  'iris classification',
  'file:///C:/Users/carnot/Desktop/mlflow/mlruns/1',
  'active',
  1704679599786,
  1704679599786)]

#### registered_models

In [27]:
# table structure
query = """
PRAGMA table_info('registered_models')
"""
result = c.execute(query)
result.fetchall()

[(0, 'name', 'VARCHAR(256)', 1, None, 1),
 (1, 'creation_time', 'BIGINT', 0, None, 0),
 (2, 'last_updated_time', 'BIGINT', 0, None, 0),
 (3, 'description', 'VARCHAR(5000)', 0, None, 0)]

In [24]:
query = """
SELECT * 
FROM registered_models
"""
result = c.execute(query)
result.fetchall()

[('iris_model', 1704679606346, 1704679606374, None)]

#### runs

In [28]:
# table structure
query = """
PRAGMA table_info('runs')
"""
result = c.execute(query)
result.fetchall()

[(0, 'run_uuid', 'VARCHAR(32)', 1, None, 1),
 (1, 'name', 'VARCHAR(250)', 0, None, 0),
 (2, 'source_type', 'VARCHAR(20)', 0, None, 0),
 (3, 'source_name', 'VARCHAR(500)', 0, None, 0),
 (4, 'entry_point_name', 'VARCHAR(50)', 0, None, 0),
 (5, 'user_id', 'VARCHAR(256)', 0, None, 0),
 (6, 'status', 'VARCHAR(9)', 0, None, 0),
 (7, 'start_time', 'BIGINT', 0, None, 0),
 (8, 'end_time', 'BIGINT', 0, None, 0),
 (9, 'source_version', 'VARCHAR(50)', 0, None, 0),
 (10, 'lifecycle_stage', 'VARCHAR(20)', 0, None, 0),
 (11, 'artifact_uri', 'VARCHAR(200)', 0, None, 0),
 (12, 'experiment_id', 'INTEGER', 0, None, 0),
 (13, 'deleted_time', 'BIGINT', 0, None, 0)]

In [25]:
query = """
SELECT * 
FROM runs
"""
result = c.execute(query)
result.fetchall()

[('4e5d585615ad4832bbaf3eeb3c9989ef',
  'handsome-shoat-904',
  'UNKNOWN',
  '',
  '',
  'carnot',
  'FINISHED',
  1704441364650,
  1704441371683,
  '',
  'active',
  'file:///C:/Users/carnot/Desktop/mlflow/mlruns/0/4e5d585615ad4832bbaf3eeb3c9989ef/artifacts',
  0,
  None),
 ('68df3105fe12435fa339f4bcbc62b355',
  'abcde',
  'UNKNOWN',
  '',
  '',
  'carnot',
  'FINISHED',
  1704679600210,
  1704679606405,
  '',
  'active',
  'file:///C:/Users/carnot/Desktop/mlflow/mlruns/1/68df3105fe12435fa339f4bcbc62b355/artifacts',
  1,
  None)]

#### model_versions

In [15]:
# table structure
query = """
PRAGMA table_info('model_versions')
"""
result = c.execute(query)
result.fetchall()

[(0, 'name', 'VARCHAR(256)', 1, None, 1),
 (1, 'version', 'INTEGER', 1, None, 2),
 (2, 'creation_time', 'BIGINT', 0, None, 0),
 (3, 'last_updated_time', 'BIGINT', 0, None, 0),
 (4, 'description', 'VARCHAR(5000)', 0, None, 0),
 (5, 'user_id', 'VARCHAR(256)', 0, None, 0),
 (6, 'current_stage', 'VARCHAR(20)', 0, None, 0),
 (7, 'source', 'VARCHAR(500)', 0, None, 0),
 (8, 'run_id', 'VARCHAR(32)', 0, None, 0),
 (9, 'status', 'VARCHAR(20)', 0, None, 0),
 (10, 'status_message', 'VARCHAR(500)', 0, None, 0),
 (11, 'run_link', 'VARCHAR(500)', 0, None, 0)]

In [16]:
query = """
SELECT * 
FROM model_versions
"""
result = c.execute(query)
result.fetchall()

[('iris_model',
  1,
  1704679606374,
  1704679606374,
  None,
  None,
  'None',
  'file:///C:/Users/carnot/Desktop/mlflow/mlruns/1/68df3105fe12435fa339f4bcbc62b355/artifacts/iris_model',
  '68df3105fe12435fa339f4bcbc62b355',
  'READY',
  None,
  None)]

#### latest_metrics

In [17]:
# table structure
query = """
PRAGMA table_info('latest_metrics')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'FLOAT', 1, None, 0),
 (2, 'timestamp', 'BIGINT', 0, None, 0),
 (3, 'step', 'BIGINT', 1, None, 0),
 (4, 'is_nan', 'BOOLEAN', 1, None, 0),
 (5, 'run_uuid', 'VARCHAR(32)', 1, None, 2)]

In [18]:
query = """
SELECT * 
FROM latest_metrics
"""
result = c.execute(query)
result.fetchall()

[('training_mean_squared_error',
  1362.2927408171518,
  1704441365061,
  0,
  0,
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('training_mean_absolute_error',
  30.825004251359697,
  1704441365061,
  0,
  0,
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('training_r2_score',
  0.7761959125801277,
  1704441365061,
  0,
  0,
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('training_root_mean_squared_error',
  36.9092500711834,
  1704441365061,
  0,
  0,
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('training_score',
  0.7761959125801277,
  1704441365097,
  0,
  0,
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('accuracy', 1.0, 1704679600256, 0, 0, '68df3105fe12435fa339f4bcbc62b355')]

#### metrics

In [19]:
# table structure
query = """
PRAGMA table_info('metrics')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'FLOAT', 1, None, 5),
 (2, 'timestamp', 'BIGINT', 1, None, 2),
 (3, 'run_uuid', 'VARCHAR(32)', 1, None, 4),
 (4, 'step', 'BIGINT', 1, "'0'", 3),
 (5, 'is_nan', 'BOOLEAN', 1, "'0'", 6)]

In [20]:
query = """
SELECT * 
FROM metrics
"""
result = c.execute(query)
result.fetchall()

[('training_mean_squared_error',
  1362.2927408171518,
  1704441365061,
  '4e5d585615ad4832bbaf3eeb3c9989ef',
  0,
  0),
 ('training_mean_absolute_error',
  30.825004251359697,
  1704441365061,
  '4e5d585615ad4832bbaf3eeb3c9989ef',
  0,
  0),
 ('training_r2_score',
  0.7761959125801277,
  1704441365061,
  '4e5d585615ad4832bbaf3eeb3c9989ef',
  0,
  0),
 ('training_root_mean_squared_error',
  36.9092500711834,
  1704441365061,
  '4e5d585615ad4832bbaf3eeb3c9989ef',
  0,
  0),
 ('training_score',
  0.7761959125801277,
  1704441365097,
  '4e5d585615ad4832bbaf3eeb3c9989ef',
  0,
  0),
 ('accuracy', 1.0, 1704679600256, '68df3105fe12435fa339f4bcbc62b355', 0, 0)]

#### params

In [21]:
# table structure
query = """
PRAGMA table_info('params')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'VARCHAR(500)', 1, None, 0),
 (2, 'run_uuid', 'VARCHAR(32)', 1, None, 2)]

In [22]:
query = """
SELECT * 
FROM params
"""
result = c.execute(query)
result.fetchall()

[('bootstrap', 'True', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('ccp_alpha', '0.0', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('criterion', 'squared_error', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('max_depth', '6', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('max_features', '3', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('max_leaf_nodes', 'None', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('max_samples', 'None', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('min_impurity_decrease', '0.0', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('min_samples_leaf', '1', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('min_samples_split', '2', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('min_weight_fraction_leaf', '0.0', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('n_estimators', '100', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('n_jobs', 'None', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('oob_score', 'False', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('random_state', 'None', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('verbose', '0', '4e5d585615ad4832bbaf3eeb3c9989ef

#### registered_model_aliases

In [23]:
# table structure
query = """
PRAGMA table_info('registered_model_aliases')
"""
result = c.execute(query)
result.fetchall()

[(0, 'alias', 'VARCHAR(256)', 1, None, 2),
 (1, 'version', 'INTEGER', 1, None, 0),
 (2, 'name', 'VARCHAR(256)', 1, None, 1)]

In [24]:
query = """
SELECT * 
FROM registered_model_aliases
"""
result = c.execute(query)
result.fetchall()

[]

#### datasets

In [25]:
# table structure
query = """
PRAGMA table_info('datasets')
"""
result = c.execute(query)
result.fetchall()

[(0, 'dataset_uuid', 'VARCHAR(36)', 1, None, 0),
 (1, 'experiment_id', 'INTEGER', 1, None, 1),
 (2, 'name', 'VARCHAR(500)', 1, None, 2),
 (3, 'digest', 'VARCHAR(36)', 1, None, 3),
 (4, 'dataset_source_type', 'VARCHAR(36)', 1, None, 0),
 (5, 'dataset_source', 'TEXT', 1, None, 0),
 (6, 'dataset_schema', 'TEXT', 0, None, 0),
 (7, 'dataset_profile', 'TEXT', 0, None, 0)]

In [26]:
query = """
SELECT * 
FROM datasets
"""
result = c.execute(query)
result.fetchall()

[('8e18e687d0f34586b32d714a86c6599a',
  0,
  'dataset',
  '545f01ac',
  'code',
  '{"tags": {"mlflow.user": "carnot", "mlflow.source.name": "C:\\\\Users\\\\carnot\\\\anaconda3\\\\envs\\\\mlflow_testing\\\\Lib\\\\site-packages\\\\ipykernel_launcher.py", "mlflow.source.type": "LOCAL"}}',
  '{"mlflow_tensorspec": {"features": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": {\\"dtype\\": \\"float64\\", \\"shape\\": [-1, 10]}}]", "targets": "[{\\"type\\": \\"tensor\\", \\"tensor-spec\\": {\\"dtype\\": \\"float64\\", \\"shape\\": [-1]}}]"}}',
  '{"features_shape": [331, 10], "features_size": 3310, "features_nbytes": 26480, "targets_shape": [331], "targets_size": 331, "targets_nbytes": 2648}'),
 ('06e9e1f627754b7fa48effe413877005',
  0,
  'dataset',
  'c8552247',
  'code',
  '{"tags": {"mlflow.user": "carnot", "mlflow.source.name": "C:\\\\Users\\\\carnot\\\\anaconda3\\\\envs\\\\mlflow_testing\\\\Lib\\\\site-packages\\\\ipykernel_launcher.py", "mlflow.source.type": "LOCAL"}}',
  '{"mlflow_tens

#### inputs

In [27]:
# table structure
query = """
PRAGMA table_info('inputs')
"""
result = c.execute(query)
result.fetchall()

[(0, 'input_uuid', 'VARCHAR(36)', 1, None, 0),
 (1, 'source_type', 'VARCHAR(36)', 1, None, 1),
 (2, 'source_id', 'VARCHAR(36)', 1, None, 2),
 (3, 'destination_type', 'VARCHAR(36)', 1, None, 3),
 (4, 'destination_id', 'VARCHAR(36)', 1, None, 4)]

In [29]:
query = """
SELECT * 
FROM inputs
"""
result = c.execute(query)
result.fetchall()

[('1aec66dfa9a94664aec349f1f83db243',
  'DATASET',
  '8e18e687d0f34586b32d714a86c6599a',
  'RUN',
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('93e1e3ed720f42448fde183ffe0cb147',
  'DATASET',
  '06e9e1f627754b7fa48effe413877005',
  'RUN',
  '4e5d585615ad4832bbaf3eeb3c9989ef')]

#### input_tags

In [31]:
# table structure
query = """
PRAGMA table_info('input_tags')
"""
result = c.execute(query)
result.fetchall()

[(0, 'input_uuid', 'VARCHAR(36)', 1, None, 1),
 (1, 'name', 'VARCHAR(255)', 1, None, 2),
 (2, 'value', 'VARCHAR(500)', 1, None, 0)]

In [30]:
query = """
SELECT * 
FROM input_tags
"""
result = c.execute(query)
result.fetchall()

[('1aec66dfa9a94664aec349f1f83db243', 'mlflow.data.context', 'train'),
 ('93e1e3ed720f42448fde183ffe0cb147', 'mlflow.data.context', 'eval')]

#### tags

In [8]:
# table structure
query = """
PRAGMA table_info('tags')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'VARCHAR(5000)', 0, None, 0),
 (2, 'run_uuid', 'VARCHAR(32)', 1, None, 2)]

In [23]:
query = """
SELECT * 
FROM tags
"""
result = c.execute(query)
result.fetchall()

[('mlflow.user', 'carnot', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('mlflow.source.name',
  'C:\\Users\\carnot\\anaconda3\\envs\\mlflow_testing\\Lib\\site-packages\\ipykernel_launcher.py',
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('mlflow.source.type', 'LOCAL', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('mlflow.autologging', 'sklearn', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('mlflow.runName', 'handsome-shoat-904', '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('estimator_name',
  'RandomForestRegressor',
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('estimator_class',
  'sklearn.ensemble._forest.RandomForestRegressor',
  '4e5d585615ad4832bbaf3eeb3c9989ef'),
 ('mlflow.log-model.history',
  '[{"run_id": "4e5d585615ad4832bbaf3eeb3c9989ef", "artifact_path": "model", "utc_time_created": "2024-01-05 07:56:05.165272", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.11.5", "env": {"conda": "conda.yaml", "virtualenv": "pytho

#### registered_model_tags

In [10]:
# table structure
query = """
PRAGMA table_info('registered_model_tags')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'VARCHAR(5000)', 0, None, 0),
 (2, 'name', 'VARCHAR(256)', 1, None, 2)]

In [12]:
query = """
SELECT * 
FROM registered_model_tags
"""
result = c.execute(query)
result.fetchall()

[]

#### experiment_tags

In [6]:
# table structure
query = """
PRAGMA table_info('experiment_tags')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'VARCHAR(5000)', 0, None, 0),
 (2, 'experiment_id', 'INTEGER', 1, None, 2)]

In [7]:
query = """
SELECT * 
FROM experiment_tags
"""
result = c.execute(query)
result.fetchall()

[]

#### model_version_tags

In [13]:
# table structure
query = """
PRAGMA table_info('model_version_tags')
"""
result = c.execute(query)
result.fetchall()

[(0, 'key', 'VARCHAR(250)', 1, None, 1),
 (1, 'value', 'VARCHAR(5000)', 0, None, 0),
 (2, 'name', 'VARCHAR(256)', 1, None, 2),
 (3, 'version', 'INTEGER', 1, None, 3)]

In [14]:
query = """
SELECT * 
FROM model_version_tags
"""
result = c.execute(query)
result.fetchall()

[]

#### alembic_version

In [9]:
# table structure
query = """
PRAGMA table_info('alembic_version')
"""
result = c.execute(query)
result.fetchall()

[(0, 'version_num', 'VARCHAR(32)', 1, None, 1)]

In [21]:
query = """
SELECT * 
FROM alembic_version
"""
result = c.execute(query)
result.fetchall()

[('7f2a7d5fae7d',)]

### Accessing the models

Noted that to load model with database is exact same as you would do in running locally or with other methods

**Three ways demonstrated here**
1. Load model
2. Model serving
3. Docker container

#### 1. Load model with API

In [6]:
query = """
SELECT name
FROM registered_models
"""
result = c.execute(query)
result.fetchall()

[('iris_model',)]

In [7]:
model_uri = "models:/iris_model/1"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [9]:
loaded_model.predict(X_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

#### 2. Model serving

**set up connection to mlflow in CLI if you haven't**

`export MLFLOW_TRACKING_URI=sqlite:///mlruns.db` This line set up connection with mlflow

`$env:MLFLOW_TRACKING_URI="sqlite:///mlruns.db"` (if you are using windows)

**serve the model on CLI**

`mlflow models serve -m "models:/iris_model/1" --port 5002`

**To use the model to make predictions, you can use the `curl` command**

`curl http://127.0.0.1:5002/invocations -H 'Content-Type: application/json' -d '{
  "dataframe_split": {
      "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]
  }
}'`

More about the curl command: https://mlflow.org/docs/latest/models.html#local-model-deployment

In [ ]:
import requests

endpoint = "http://localhost:5002/invocations" # make sure the port number matches the one you input in terminal
data = {
    "dataframe_split": {  
        "data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]   
    }
}
# do a post request
response = requests.post(endpoint, json=data)
print(response.json())

#### 3. Docker

(noted that you will need docker installed on your computer to use this feature https://www.docker.com/products/docker-desktop/)


**Instead of using serve to run the model, you can build a docker image instead** 

`mlflow models build-docker --model-uri "models:/iris_model/1" --name "iris"` This line build the docker image

`docker run -p 5002:8080 iris` Run the docker container on the respective port

**To use the model, you can use the exact same command as you do in model serving**

`curl http://127.0.0.1:8080/invocations -H 'Content-Type: application/json' -d '{"dataframe_split": {"data": [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3]]}}'`